In [1]:
import os
import openai
# import tiktoken
import pandas as pd

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# OPENAI_API_KEY=''
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

EMBEDDING_MODEL = 'text-embedding-ada-002'
GPT_MODEL = 'gpt-3.5-turbo-0301'
MAX_MODEL_TOKEN = 8000 # max: 8191

In [3]:
# encoding = tiktoken.encoding_for_model(GPT_MODEL)

In [4]:
openai.api_key = OPENAI_API_KEY

In [5]:
df = pd.read_csv('./data/raw_content.csv')
df.head(5)

,url,content
0,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のサイズ商品のサイズは、商品ページ下部にある「サイズ」の項目をご覧ください。当サイトでは...
1,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のカラー・デザイン商品の画像は、できる限り実際の商品に近づけた状態で掲載をしております。...
2,https://zozo.jp/_help/default.html?id=5ece47a3...,商品の販売価格サイトの表示価格は、販売ショップがZOZOTOWNで販売するために設定した価格...
3,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSEDの取扱い商品ZOZOUSEDでは、買取サービスや買い替え割・いつでも買い替え...
4,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSED商品の掲載画像撮影時の照明やカメラの角度、お客様の端末設定により、実際の色味...


In [6]:
def get_embedding(text):
    return openai.Embedding.create(input=[text], model=EMBEDDING_MODEL)['data'][0]['embedding']

In [7]:
results = []
for i in range(len(df)):
    row = df.iloc[i]
    url = row.get('url')
    content = row.get('content')
    embedding = get_embedding(
        row.get('content')[:MAX_MODEL_TOKEN]
    )

    results += [{
        'url': url,
        'content': content,
        'embedding': embedding,
    }]

In [8]:
df_new = pd.DataFrame(results)
df_new.head(5)

,url,content,embedding
0,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のサイズ商品のサイズは、商品ページ下部にある「サイズ」の項目をご覧ください。当サイトでは...,"[-0.012270285747945309, 0.009278957732021809, ..."
1,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のカラー・デザイン商品の画像は、できる限り実際の商品に近づけた状態で掲載をしております。...,"[-0.012848308309912682, -0.0011484753340482712..."
2,https://zozo.jp/_help/default.html?id=5ece47a3...,商品の販売価格サイトの表示価格は、販売ショップがZOZOTOWNで販売するために設定した価格...,"[-0.015740083530545235, -0.006914299912750721,..."
3,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSEDの取扱い商品ZOZOUSEDでは、買取サービスや買い替え割・いつでも買い替え...,"[-0.01777864620089531, -0.008320022374391556, ..."
4,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSED商品の掲載画像撮影時の照明やカメラの角度、お客様の端末設定により、実際の色味...,"[-0.004656823817640543, -0.005725925322622061,..."


In [9]:
df_new.to_csv('./data/embedding_content.csv', index=False)